In [ ]:
import tensorflow.keras
import numpy as np
import cv2
from pyKamipi.pibot import *

In [ ]:
kamibot= KamibotPi('COM5', 57600)

In [ ]:
model = tensorflow.keras.models.load_model('keras_model.h5')

In [ ]:
capture = cv2.VideoCapture(0)

In [ ]:
capture.set(cv2.CAP_PROP_FRAME_WIDTH, 1280)
capture.set(cv2.CAP_PROP_FRAME_HEIGHT, 960)

In [ ]:
def preprocessing(frame):

    size = (224, 224)
    frame_resized = cv2.resize(frame, size)
    
    frame_normalized = (frame_resized.astype(np.float32) / 127.0) - 1

    frame_reshaped = frame_normalized.reshape((1, 224, 224, 3))
    
    return frame_reshaped

In [ ]:
def move_country(arr):
    global cur_loc
    distance = (location[arr][0]-cur_loc[0], location[arr][1]-cur_loc[1])
    
    if distance[0] > 0:
        kamibot.move_forward(distance[0])
    elif distance[0] < 0:
        kamibot.turn_back()
        kamibot.move_forward(abs(distance[0]))
    move_updown(distance)
    
    cur_loc = location[arr]
    arrive_melody()

In [ ]:
def move_updown(distance):
    if distance[0] >= 0:
        if distance[1] > 0:
            kamibot.turn_left()    
            kamibot.move_forward(distance[1])
            kamibot.turn_right()
        else:
            kamibot.turn_right()    
            kamibot.move_forward(abs(distance[1]))
            kamibot.turn_left()
    else:
        if distance[1] > 0:
            kamibot.turn_right()    
            kamibot.move_forward(abs(distance[1]))
            kamibot.turn_right()
        else:
            kamibot.turn_left()    
            kamibot.move_forward(distance[1])
            kamibot.turn_left()

In [ ]:
def arrive_melody():
    melody = [52, 53, 55]
    for i in melody:
        kamibot.melody(i,0.35)

In [ ]:
location = {'uk':[1,2], 'turkey':[2,1], 'korea':[4,1]}
cur_loc = [1,1]

In [ ]:
while True:
    ret, frame = capture.read()

    if cv2.waitKey(100) > 0: 
        break

    prediction = model.predict(preprocessing(frame))
    
    if (prediction[0,0] > 0.8):
        cv2.putText(frame, 'uk', (0, 25), 0, 1, (0, 0, 0))
        kamibot.turn_led(0, 255, 0)
        move_country('uk')

    elif (prediction[0,1] > 0.8):
        cv2.putText(frame, 'korea', (0, 25), 0, 1, (0, 0, 0))
        kamibot.turn_led(255, 0, 0)
        move_country('korea')
        
    elif (prediction[0,2] > 0.8):
        cv2.putText(frame, 'turkey', (0, 25), 0, 1, (0, 0, 0))
        kamibot.turn_led(0, 0, 255)
        move_country('turkey')
        
    else:
        cv2.putText(frame, 'None', (0, 25), 0, 1, (0, 0, 0))
        kamibot.turn_led(255, 255, 255)

    cv2.imshow("VideoFrame", frame)
    
cv2.destroyAllWindows()

In [ ]:
# def move(count):
#     cnt_cross = 0
#     check = 0
#     while cnt_cross < count:
#         left, center, right = kamibot.get_line_sensor()  # 왼쪽, 중앙, 오른쪽 센서 값 읽기
#         if center==1 and left==1 and right==1:
#             while right==1:
#                 left, center, right = kamibot.get_line_sensor()  # 왼쪽, 중앙, 오른쪽 센서 값 읽기
#                 if left == 1:
#                     kamibot.go_forward_speed(0, 20)
#                 elif center == 1:
#                     kamibot.go_forward_speed(20, 0)
#                 line_tracing()
#             cnt_cross += 1
#             print(cnt_cross)
#             #소리 추가
#         else:
#             line_tracing()
            
#     kamibot.move_forward_unit(1, "-l") # 10cm
#     kamibot.delay(1)
#     kamibot.stop()

In [ ]:
# def line_tracing():
#     left, center, right = kamibot.get_line_sensor()  # 왼쪽, 중앙, 오른쪽 센서 값 읽기
#     if left == 0:
#         kamibot.go_forward_speed(0, 20)
#     elif center == 0:
#         kamibot.go_forward_speed(20, 0)

In [ ]:
# def move_up(count):
#     kamibot.go_dir_speed("f", 40, "b", 40)
#     kamibot.delay(1)
#     move(count)
#     kamibot.go_dir_speed("b", 40, "f", 40)
#     kamibot.delay(1)
#     kamibot.stop()